In [1]:
from sklearn.datasets import load_wine

In [2]:
wine = load_wine()

In [27]:
x = wine.data
y = wine.target

In [28]:
x.shape, y.shape

((178, 13), (178,))

In [29]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

### 조건:  Functional API
1. training set / test set / validation set 구분하기!
2. scaling(min_max, standardization 둘 중!)
3. y에 대한 전처리
4. 히든 레이더 최소 2개 쓰기 (노드와 엑티베이션 자유!)
5. early stopping, validation_data

In [30]:
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state= 2023)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2023)

In [32]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((144, 13), (16, 13), (18, 13), (144,), (16,), (18,))

In [33]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_m, scaler_s = MinMaxScaler(), StandardScaler()


In [ ]:
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [34]:
import pandas as pd

In [35]:
x_train_m = scaler_m.fit_transform(x_train)
pd.DataFrame(x_train_m, columns = wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000,144.000000
mean,0.476599,0.293873,0.439542,0.434176,0.369170,0.467864,0.371469,0.426887,0.373861,0.289860,0.492669,0.516153,0.335442
std,0.241979,0.225709,0.175955,0.177628,0.167823,0.220203,0.209416,0.230316,0.173968,0.204613,0.234913,0.262204,0.231661
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.274242,0.146640,0.344771,0.305851,0.234568,0.273276,0.196730,0.245283,0.267350,0.133215,0.306701,0.288462,0.151079
50%,0.484848,0.194501,0.431373,0.414894,0.345679,0.494828,0.401899,0.396226,0.362776,0.249556,0.494845,0.576923,0.279856
75%,0.680303,0.429735,0.550654,0.521277,0.459877,0.644828,0.538502,0.566038,0.485804,0.380551,0.662371,0.717949,0.500899
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
x_train_s = scaler_s.fit_transform(x_train)
pd.DataFrame(x_train_s, columns = wine.feature_names).describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02,1.440000e+02
mean,2.146431e-15,3.022274e-16,1.406282e-15,-1.881211e-16,4.749287e-16,-1.094803e-15,2.467162e-17,-8.018277e-17,8.264994e-16,8.465451e-16,-1.005369e-15,-9.806970e-16,-7.401487e-17
std,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00,1.003490e+00
min,-1.976465e+00,-1.306545e+00,-2.506755e+00,-2.452833e+00,-2.207437e+00,-2.132107e+00,-1.780024e+00,-1.859949e+00,-2.156522e+00,-1.421566e+00,-2.104557e+00,-1.975384e+00,-1.453037e+00
25%,-8.391772e-01,-6.545931e-01,-5.404900e-01,-7.249568e-01,-8.048480e-01,-8.867592e-01,-8.373240e-01,-7.912490e-01,-6.143799e-01,-7.682367e-01,-7.944079e-01,-8.714040e-01,-7.986053e-01
50%,3.420939e-02,-4.418032e-01,-4.659396e-02,-1.089313e-01,-1.404639e-01,1.228761e-01,1.458139e-01,-1.335875e-01,-6.393999e-02,-1.976624e-01,9.297043e-03,2.325756e-01,-2.407805e-01
75%,8.447624e-01,6.040366e-01,6.336779e-01,4.920691e-01,5.423753e-01,8.064423e-01,8.003966e-01,6.062817e-01,6.457181e-01,4.447782e-01,7.249247e-01,7.722990e-01,7.167148e-01
max,2.170551e+00,3.139406e+00,3.196346e+00,3.196571e+00,3.772020e+00,2.425002e+00,3.011825e+00,2.497059e+00,3.611725e+00,3.482760e+00,2.167190e+00,1.851746e+00,2.878676e+00


In [37]:
x_val_s = scaler_s.transform(x_val)
x_test_s = scaler_s.transform(x_test)

In [38]:
x_train.min(), x_train.max(), x_val.min(), x_val.max(), x_test.min(), x_test.max()

(0.13, 1680.0, 0.2, 1120.0, 0.14, 1285.0)

In [39]:
# column별로 min, max 값이 다르기 때문에 column 별로 스케일링이 필요하다.
# max_num = x_train.max()
# min_num = x_train.min()

# x_train = (x_train + min_num) / (max_num + min_num)
# x_val = (x_val + min_num) / (max_num + min_num)
# x_test = (x_test + min_num) / (max_num + min_num)

In [40]:
# x_train.min(), x_train.max(), x_val.min(), x_val.max(), x_test.min(), x_test.max()

In [41]:
from tensorflow.keras.utils import to_categorical

''

In [42]:
len_y = len(set(y_train))

y_train = to_categorical(y_train, len_y)
y_val = to_categorical(y_val, len_y)
y_test = to_categorical(y_test, len_y)

In [52]:
from tensorflow.keras.callbacks import EarlyStopping

In [53]:
es = EarlyStopping(monitor = 'val_loss',            # 관측대상
                   min_delta = 0,                   # 임계값
                   patience = 5,                    # 관측 대상의 성능 개선이 발생하지 않을 때, 얼마나 더 지켜볼 것인지
                   verbose = 1,
                   restore_best_weights= True)      # 최적의 epochs 가중치로 적용

In [54]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((144, 13), (16, 13), (18, 13), (144, 3), (16, 3), (18, 3))

In [60]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 사슬처럼 엮기
il = keras.layers.Input(shape = (13, ))
hl = keras.layers.Dense(64, activation = 'relu', name= 'hidden1')(il)
hl = keras.layers.Dense(32, activation = 'relu', name= 'hidden2')(hl)
ol = keras.layers.Dense(3, activation = 'softmax', name= 'output')(hl)

# 3. 모델 시작/끝 지정
model = keras.models.Model(il, ol)

# 4. 컴파일
model.compile(loss = keras.losses.categorical_crossentropy,
              metrics= ['accuracy'],
              optimizer= 'adam')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 hidden1 (Dense)             (None, 64)                896       
                                                                 
 hidden2 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 3)                 99        
                                                                 
Total params: 3,075
Trainable params: 3,075
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(x_train_s, y_train,
          epochs = 50,
          verbose = 1,
          callbacks = [es],
          validation_data = (x_val_s, y_val))

Epoch 1/50
5/5 [==============================] - 1s 70ms/step - loss: 0.9508 - accuracy: 0.5278 - val_loss: 0.7353 - val_accuracy: 0.7500
Epoch 2/50
5/5 [==============================] - 0s 13ms/step - loss: 0.8062 - accuracy: 0.7222 - val_loss: 0.6185 - val_accuracy: 0.8750
Epoch 3/50
5/5 [==============================] - 0s 15ms/step - loss: 0.6801 - accuracy: 0.8819 - val_loss: 0.5135 - val_accuracy: 0.9375
Epoch 4/50
5/5 [==============================] - 0s 16ms/step - loss: 0.5680 - accuracy: 0.9306 - val_loss: 0.4239 - val_accuracy: 0.9375
Epoch 5/50
5/5 [==============================] - 0s 16ms/step - loss: 0.4703 - accuracy: 0.9583 - val_loss: 0.3471 - val_accuracy: 1.0000
Epoch 6/50
5/5 [==============================] - 0s 19ms/step - loss: 0.3854 - accuracy: 0.9653 - val_loss: 0.2848 - val_accuracy: 1.0000
Epoch 7/50
5/5 [==============================] - 0s 19ms/step - loss: 0.3163 - accuracy: 0.9722 - val_loss: 0.2332 - val_accuracy: 1.0000
Epoch 8/50
5/5 [===========

In [62]:
y_pred = model.predict(x_test_s)

print(y[:10])
print(y_pred.round(0).reshape(-1,)[:10])

1/1 [==============================] - 0s 52ms/step
[0 0 0 0 0 0 0 0 0 0]
[0. 0. 1. 0. 1. 0. 0. 0. 1. 0.]
